<a href="https://colab.research.google.com/github/amien1410/amien-scrapers/blob/main/Trustpilot_Reviews_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Modules and Functions (Click play button below)

In [ ]:
# @title
import json
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from google.colab import files
from urllib.parse import urlparse

# Initiate headers
headers = {
    "Host": "www.trustpilot.com",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:126.0) Gecko/20100101 Firefox/126.0",
    "Accept": "*/*",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Referer": "https://www.trustpilot.com/review/prodigyfinance.com",
    "x-nextjs-data": "1",
    "Connection": "keep-alive",
    "Cookie": "OptanonConsent=isGpcEnabled=0&datestamp=Sat+May+18+2024+08%3A52%3A55+GMT%2B0800+(China+Standard+Time)&version=6.28.0&isIABGlobal=false&hosts=&consentId=db739580-2c05-49ff-a98a-516a3dbcbef9&interactionCount=2&landingPath=NotLandingPage&groups=C0001%3A1%2CC0002%3A1%2CC0003%3A1%2CC0004%3A1&AwaitingReconsent=false; TP.uuid=2c74dd4f-b48b-4218-90da-bcfb1aa1f011; _hjSessionUser_391767=eyJpZCI6IjgzOGQwNTUxLTIzY2QtNTExOS1iY2QyLTFhNGE1NzZjY2UwNSIsImNyZWF0ZWQiOjE3MTU5OTMxNzc5MTAsImV4aXN0aW5nIjp0cnVlfQ==; _hjSession_391767=eyJpZCI6IjM4MmVkN2M5LTI2YTEtNGM1Mi1iMmI5LTY0ZjJmNmNiNGU2YyIsImMiOjE3MTU5OTMxNzc5MTIsInMiOjEsInIiOjEsInNiIjowLCJzciI6MCwic2UiOjAsImZzIjoxLCJzcCI6MX0=; ajs_anonymous_id=7ac94ec7-630d-4359-9c7a-e8c16de4266f; amplitude_idundefinedtrustpilot.com=eyJvcHRPdXQiOmZhbHNlLCJzZXNzaW9uSWQiOm51bGwsImxhc3RFdmVudFRpbWUiOm51bGwsImV2ZW50SWQiOjAsImlkZW50aWZ5SWQiOjAsInNlcXVlbmNlTnVtYmVyIjowfQ==; amplitude_id_67f7b7e6c8cb1b558b0c5bda2f747b07trustpilot.com=eyJkZXZpY2VJZCI6IjM0OGFkZDNkLTc5N2UtNDg2Zi1iZTg5LWI3NDhkNjI0MTU2NlIiLCJ1c2VySWQiOm51bGwsIm9wdE91dCI6ZmFsc2UsInNlc3Npb25JZCI6MTcxNTk5MzE4Njc3MCwibGFzdEV2ZW50VGltZSI6MTcxNTk5MzU2ODA0OCwiZXZlbnRJZCI6NSwiaWRlbnRpZnlJZCI6MCwic2VxdWVuY2VOdW1iZXIiOjV9; _gcl_au=1.1.1883098336.1715993193; _hjHasCachedUserAttributes=true; _tt_enable_cookie=1; _ttp=J87slJZtgAwRD7KZmgwdj1bX5NN; _ga_11HBWMC274=GS1.1.1715993199.1.1.1715993569.34.0.0; _ga=GA1.1.1321321732.1715993200; _hjUserAttributesHash=e924cae0540e45e3eeaece5bbca58716; OptanonAlertBoxClosed=2024-05-18T00:52:55.568Z",
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "same-origin",
    "Priority": "u=1",
    "TE": "trailers"
}

# Function to validate the URL
def is_valid_url(url):
    parsed_url = urlparse(url)
    return parsed_url.scheme == "https" and parsed_url.netloc == "www.trustpilot.com" and parsed_url.path.startswith("/review/")

# Function to get all reviews
def scrapeReviews(baseUrl):
    page = 1
    all_reviews = []

    while True:
        try:
            # Construct URL with page parameter (except for first page)
            url = baseUrl if page == 1 else f"{baseUrl}?page={page}"
            print(f"Scraping reviews from: {url}")

            # Get reviews for this page
            res = requests.get(url, headers=headers)
            res.raise_for_status()
            soup = BeautifulSoup(res.text, 'html.parser')

            # Find the script tag with the specific id and type
            script_tag = soup.find('script', {'id': '__NEXT_DATA__', 'type': 'application/json'})

            # Extract the JSON string inside the script tag and convert it to a Python dictionary
            if script_tag:
                json_string = script_tag.string.strip()
                json_data = json.loads(json_string)
                reviews = json_data.get('props', {}).get('pageProps', {}).get('reviews', [])

            # Check if there are any reviews on this page
            if not reviews:
                break

            # Add reviews to the list
            print(f"Total comments on this page: {len(reviews)}")
            all_reviews.extend(reviews)

            # Check if there are less than 20 reviews on this page
            if len(reviews) < 20:
                break

            # Increment page number
            page += 1
        except requests.RequestException as e:
            print(f"Error occurred: {e}. Retrying in 5 seconds...")
            time.sleep(5)
            continue  # Retry from the last page

    return all_reviews

# Function to convert url into filename
def getFilename(url):
    parsed_url = urlparse(url)
    filename = parsed_url.path.split('/')[-1]
    return filename

# Main function
def main():
    result = []
    url = input("Enter your Trustpilot url here: ")

    while not is_valid_url(url):
        print("Invalid URL. The URL must be in the format: https://www.trustpilot.com/review/[company]")
        url = input("Enter your Trustpilot url here: ")

    while True:
        try:
            allReviews = scrapeReviews(url)
            break
        except requests.RequestException as e:
            print(f"Network error: {e}. Please check your internet connection or the URL and try again.")
            url = input("Enter your Trustpilot url here: ")

    filename = getFilename(url)
    for data in allReviews:
        result.append({
            "reviewId": data["id"],
            "title": data["title"],
            "text": data["text"],
            "rating": data["rating"],
            "isVerified": data["labels"]["verification"]["isVerified"],
            "likes": data["likes"],
            "publishedDate": data["dates"]["publishedDate"],
            "consumer name": data["consumer"]["displayName"],
            "countryCode": data["consumer"]["countryCode"],
            "reply": data["reply"]["message"] if data["reply"] else None
        })
    df = pd.DataFrame(result)
    df.to_excel(f"{filename}.xlsx")
    # files.download(f"{filename}.xlsx")
    return df

# Run The Function (Click play button below), Insert Your Url into the input field, Wait until it finished and download the result automatically (Or you can download manually the file from the left panel)

In [ ]:
# Click the play button on the left to rerun the script https://www.trustpilot.com/review/prodigyfinance.com
main()